In [47]:
import pandas as pd
import requests
import ast


# Pull in data from data.sfgov API

In [82]:
url = 'https://data.sfgov.org/resource/j2j3-acqj.json'
response = requests.get(url).json()
df = pd.DataFrame(response)
df


,parcel_s,building_name,building_address,postal_code,full_address,property_type,pim_link,benchmark_2020_status,floor_area,year_built,...,_2012_weather_normalized_site_eui_kbtu_ft2,_2012_weather_normalized_source_eui_kbtu_sq_ft,_2012_percent_better_than_national_median_source_eui,_2011_energy_star_score,_2011_site_eui_kbtu_ft2,_2011_percent_better_than_national_median_site_eui,_2011_weather_normalized_site_eui_kbtu_ft2,_2011_source_eui_kbtu_ft2,_2011_percent_better_than_national_median_source_eui,_2011_weather_normalized_source_eui_kbtu_ft2
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,"{'latitude': '37.806603', 'longitude': '-122.4...",Cannabis Business,{'url': 'http://propertymap.sfplanning.org/?&s...,Upcoming,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,"{'latitude': '37.803053', 'longitude': '-122.4...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,99333,1916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0105/019,350 UNION ST,350 UNION ST,94133,"{'latitude': '37.80093', 'longitude': '-122.40...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Insufficient Data,85460,1965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,"{'latitude': '37.796419', 'longitude': '-122.4...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,67734,1976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,"{'latitude': '37.797422', 'longitude': '-122.4...",Commercial,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,33652,2002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0670/013,1320 VAN NESS AVE,1320 VAN NESS AVE,94109,"{'latitude': '37.787809', 'longitude': '-122.4...",Commercial,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,106827,1909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,0084/008,Interland - 1265 Battery Street,1265 BATTERY ST,94111,"{'latitude': '37.802923', 'longitude': '-122.4...",Commercial,{'url': 'http://propertymap.sfplanning.org/?&s...,Complied,94111,1982,...,53.2,125.9,NaN,87,49.6,-40.8,47.8,NaN,NaN,NaN
997,0298/007,Fitzgerald Hotel,620 POST ST,94109,"{'latitude': '37.787814', 'longitude': '-122.4...",Commercial,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,13104,1910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,0346/015,449 TURK ST,449 TURK ST,94102,"{'latitude': '37.782492', 'longitude': '-122.4...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,114272,1907,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fix two issues

### Split latitude, longitude, and human-readable address from full_address

In [83]:
df = pd.concat([df.drop(['full_address'], axis=1), df['full_address'].apply(pd.Series)], axis=1)



Drop NaN column

In [84]:
df.drop(0, axis=1, inplace=True)


Do the same with the resulting human_address... which is actually a string, not a dict :(

In [85]:
df = pd.concat([df.drop(['human_address'], axis=1), df['human_address'].fillna(value='1').apply(lambda x: ast.literal_eval(x)).apply(pd.Series)], axis=1)

Drop NaN column (again) and zip (since postal_code already exists)

In [86]:
df.drop(0, axis=1, inplace=True)
df.drop('zip', axis=1, inplace=True)

### Change all columns (except state) to Title Case

In [90]:
df = pd.concat([df.iloc[:, :-1].apply(lambda x: x.str.title()), df.iloc[:, -1:]], axis=1)